In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path='../data/shakespeare.txt'

In [3]:
text=open(path,'r').read()

In [5]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [6]:
vocab=sorted(set(text))

In [7]:
len(vocab)

84

In [8]:
char_to_ind={ind:char for char,ind in enumerate(vocab)}

In [9]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [10]:
ind_to_char=np.array(vocab)

In [11]:
encoded=np.array([char_to_ind[s] for s in text])

In [13]:
import tensorflow as tf

In [14]:
char_dataset=tf.data.Dataset.from_tensor_slices(encoded)

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [15]:
seq_len=120

In [16]:
sequence=char_dataset.batch(seq_len+1,drop_remainder=True)

In [23]:
def seq(seqs):
    input=seqs[:-1]
    target=seqs[1:]
    return input,target

In [24]:
datasets=sequence.map(seq)

In [25]:
batch_size=128

In [26]:
dataset=datasets.batch(batch_size,drop_remainder=True).shuffle(buffer_size=10000)

In [27]:
dataset

<_ShuffleDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [28]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [29]:
def sparse(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Embedding

In [31]:
batch_size=128
vocab_size=len(vocab)
rnn_neurons=1026
embeddings=64

In [32]:
def create_model(batch_size,vocab_size,rnn_neurons,embeddings):
    model=Sequential()
    model.add(Embedding(vocab_size,embeddings,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,stateful=True,return_sequences=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam',loss=sparse)
    return model

In [33]:
model=create_model(batch_size=batch_size,vocab_size=vocab_size,rnn_neurons=rnn_neurons,embeddings=embeddings)

In [34]:
model.fit(dataset,epochs=30)

Epoch 1/30


2023-04-14 12:53:45.092154: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  8/351 [..............................] - ETA: 1:54 - loss: 4.3372

KeyboardInterrupt: 

In [35]:
from tensorflow.keras.models import load_model

In [38]:
model=create_model(batch_size=1,vocab_size=vocab_size,rnn_neurons=rnn_neurons,embeddings=embeddings)
model.load_weights('shakespeare_gen.h5')
model.build(tf.TensorShape([batch_size,None]))

In [41]:
def generate_text(model,start_seed,gen_max,temp=1.0):
    input_eval=[char_to_ind[s] for s in start_seed]
    input_eval=tf.expand_dims(input_eval,0)
    num_generate=gen_max
    text_generated=[]
    temparature=temp
    model.reset_states()
    for i in range(num_generate):
        predictions=model(input_eval)
        predictions=predictions/temparature
        predictions=tf.squeeze(predictions,0)
        pred_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        text_generated.append(ind_to_char[pred_id])
        input_eval=tf.expand_dims([pred_id],0)
    return start_seed+''.join(text_generated)

    


In [43]:
print(generate_text(model,'Flower',1000))

Flower,
    To-day partake which I have; it holds nothing living
    ese; against God forbid-
  PASTOR. You are dinablet an epitheat,
    a husband than the censure of my mind, I wake him.
  FLORIZEL. I rare t gentle Duke, go cross this work
    But that it promises; and having his physic
    With tears and baser accurst and our funially been at Belmont.                 [Kisses him]
    We are not yet to waxen on brave betters and limbs the night
    As charms are loath to sell already feel.
    On Thine she loves.  
    I do not cheat you.
    What, not thy gratis as in a text?
    Hath no true mistress!
    What was his quarrel?
  ABHOLUS. Risenera is secus,
    That, if a great heart come to our command
    'Tis not enough of love; and that the grace
    As doubting here but mine opinion shook
    The foot and staveright with thee of advantages,  
    So soon as mine are heartify.
    To sign,
    And kiss thee leave and let no conscience may
    An it eaten here.
    Never, my lord